In [16]:
import os
import pandas as pd
import csv

import seaborn as sns
import matplotlib.pyplot as plt
from statannot import add_stat_annotation

from temp_funcs import path, csv_path, path_jem, \
read_file, merge_dataframes, drop_cols, drop_nans, filter_date_range, \
filter_df, create_container_col, create_cond_df, create_container_df

In [17]:
path_morph = os.path.join(csv_path, "short_tall_morph.csv")

In [18]:
morph_list = read_file(path_morph)

INFO:temp_funcs:Read file in as a pandas dataframe


In [19]:
morph_list["Morpho type"].unique()

array(['SHORT', 'Short', 'ShORT', 'TALL'], dtype=object)

In [20]:
morph_list["Projection Class"].unique()

array(['IT', 'CT', nan], dtype=object)

In [21]:
morph_list

,Cell Specimen Id,Morpho type,VISp Tree Mapping,Projection Class
0,836811672,SHORT,L6 IT VISp Penk Col27a1,IT
1,904005886,SHORT,L6 CT VISp Ctxn3 Brinp3,CT
2,904059133,SHORT,L6 IT VISp Col18a1,IT
3,977013731,SHORT,L6 IT VISp Col18a1,IT
4,899505310,SHORT,L6 IT VISp Col18a1,IT
5,901060224,SHORT,L6 IT VISp Col18a1,IT
6,926754976,SHORT,L6 IT VISp Col18a1,IT
7,929547484,SHORT,L6 IT VISp Penk Col27a1,IT
8,966772916,SHORT,L6 IT VISp Col18a1,IT
9,646574007,SHORT,L6 CT VISp Ctxn3 Sla,CT
